In [38]:
import os
import io
import pandas as pd
from pandas import DataFrame
import sys
import numpy as np
import json
import pickle
import jsonpickle
import time
import random

import google.cloud
from google.cloud import datastore
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

from sqlalchemy import *
import sqlalchemy
import sqlalchemy.schema
from sqlalchemy.types import *

from IPython.display import *
from pandas.io.json import *
# gcloud auth application-default login

In [39]:
#df = pd.read_pickle('/home/jm/Documents/testingStuff/my_file.pkl')
engine = create_engine('mysql+pymysql://root:@localhost/sawi_tweets?charset=utf8mb4', encoding='utf8', echo = False)
df = pd.read_sql_table("sawi_tweets_historical", con = engine)

/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_1` CHECK (`user_verified` in (0,1)),'
  util.warn("Unknown schema content: %r" % line)
/usr/local/lib/python3.5/dist-packages/sqlalchemy/dialects/mysql/reflection.py:56: SAWarning: Unknown schema content: '  CONSTRAINT `CONSTRAINT_2` CHECK (`RT` in (0,1))'
  util.warn("Unknown schema content: %r" % line)


In [40]:
df.head()

,index,created_at,favorite_count,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweet_count,screen_name,text,tweet_id,...,user_favorites_count,user_followers_count,user_friends_count,user_id,user_location,user_name,user_statuses_count,user_time_zone,user_verified,RT
0,0,2017-09-24 23:54:27,2,None,None,None,0,AmanteTech,Would next #Apple #iOS update cause your exist...,912102964881367040,...,2133,635,632,869739471570325504,"Sydney, New South Wales",AmanteTech,6314,Pacific Time (US & Canada),0,0
1,1,2017-09-24 23:30:33,0,None,None,None,84,supermeg500,RT @MegsSkunkey: iPhone 8 giveaway!!\nhttps://...,912096951532847104,...,61,361,4512,819370348420284417,,Giveaways~,402,None,0,1
2,2,2017-09-24 22:45:06,0,None,None,None,1860,botlamaya,RT @Sudathchamin: iPhone X new price?Ur left k...,912085514282553344,...,10,617,274,941255144,Colombo,Bot Lamaya [beta],51626,New Delhi,0,1
3,3,2017-09-24 22:45:06,0,None,None,None,1860,hikma_5000,RT @Sudathchamin: iPhone X new price?Ur left k...,912085512776798213,...,1388,143,113,791115735200182272,"Mansfield, TX",Hikma🌹🌹🌹,5432,Pacific Time (US & Canada),0,1
4,4,2017-09-24 22:36:56,0,None,None,None,4,Joey2031,RT @BoinkHunter: Beautiful device. \n#iPhoneX ...,912083456552783873,...,853,66,75,705859071044423680,,Joey,646,None,0,1


In [41]:
df.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,10546.000000,10546.000000,10546.000000,1.054600e+04,10546.000000,1.054600e+04,10546.000000,1.054600e+04,1.054600e+04,10546.000000,10546.000000
mean,48.537550,0.380334,3118.853689,9.138852e+17,8046.799734,7.834730e+03,2100.543049,2.119505e+17,4.355129e+04,0.012232,0.479992
std,28.807077,4.187786,12053.180200,7.966790e+14,24497.379598,6.408401e+04,14779.896690,3.631225e+17,1.299117e+05,0.109926,0.499623
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.172000e+03,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.139137e+17,66.000000,7.900000e+01,116.000000,2.857782e+08,1.215000e+03,0.000000,0.000000
50%,48.000000,0.000000,1.000000,9.142196e+17,928.000000,3.670000e+02,351.000000,2.175360e+09,7.050500e+03,0.000000,0.000000
75%,73.000000,0.000000,160.000000,9.142328e+17,5777.750000,1.420000e+03,1070.750000,7.066071e+17,3.354300e+04,0.000000,1.000000
max,99.000000,272.000000,142109.000000,9.142450e+17,589799.000000,1.906004e+06,851643.000000,9.142442e+17,2.196412e+06,1.000000,1.000000


In [42]:
df.count()

index                      10546
created_at                 10546
favorite_count             10546
in_reply_to_screen_name      926
in_reply_to_status_id        750
in_reply_to_user_id          926
retweet_count              10546
screen_name                10546
text                       10546
tweet_id                   10546
tweet_location                27
user_created_at            10546
user_description           10546
user_favorites_count       10546
user_followers_count       10546
user_friends_count         10546
user_id                    10546
user_location              10546
user_name                  10546
user_statuses_count        10546
user_time_zone              6267
user_verified              10546
RT                         10546
dtype: int64

In [43]:
dfs = df.drop_duplicates(subset = 'tweet_id', inplace=False)

In [44]:
dfs.describe()

,index,favorite_count,retweet_count,tweet_id,user_favorites_count,user_followers_count,user_friends_count,user_id,user_statuses_count,user_verified,RT
count,9960.000000,9960.000000,9960.000000,9.960000e+03,9960.000000,9.960000e+03,9960.000000,9.960000e+03,9.960000e+03,9960.000000,9960.000000
mean,48.701707,0.384839,3299.750100,9.139715e+17,8378.513253,8.118898e+03,2128.830924,2.012717e+17,4.465687e+04,0.012149,0.475502
std,28.799710,4.298809,12378.925274,7.159697e+14,25060.247992,6.587171e+04,15144.793165,3.557029e+17,1.303729e+05,0.109555,0.499425
min,0.000000,0.000000,0.000000,9.102936e+17,0.000000,0.000000e+00,0.000000,2.172000e+03,1.000000e+00,0.000000,0.000000
25%,24.000000,0.000000,0.000000,9.139159e+17,73.000000,8.800000e+01,126.000000,2.869892e+08,1.356750e+03,0.000000,0.000000
50%,48.000000,0.000000,1.000000,9.142208e+17,1009.000000,3.770000e+02,362.000000,1.937846e+09,7.652000e+03,0.000000,0.000000
75%,73.000000,0.000000,219.000000,9.142334e+17,6217.500000,1.431250e+03,1081.250000,4.850154e+09,3.546600e+04,0.000000,1.000000
max,99.000000,272.000000,142109.000000,9.142450e+17,589799.000000,1.906004e+06,851643.000000,9.142442e+17,2.196412e+06,1.000000,1.000000


In [45]:
dfs.count()

index                      9960
created_at                 9960
favorite_count             9960
in_reply_to_screen_name     926
in_reply_to_status_id       750
in_reply_to_user_id         926
retweet_count              9960
screen_name                9960
text                       9960
tweet_id                   9960
tweet_location               22
user_created_at            9960
user_description           9960
user_favorites_count       9960
user_followers_count       9960
user_friends_count         9960
user_id                    9960
user_location              9960
user_name                  9960
user_statuses_count        9960
user_time_zone             5978
user_verified              9960
RT                         9960
dtype: int64